CNN LSTM Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import keras
import keras.backend as K

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
import tensorflow as tf

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, MaxPool1D, BatchNormalization, TimeDistributed
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.models import Sequential

In [3]:
df = pd.read_csv("Preprocessed_Consumer_Data.csv")

In [4]:
# df = pd.read_csv("Synthetic_Consumer_Data.csv")

In [5]:
df.drop(["Unnamed: 0"], inplace=True, axis=1)
df.shape

(32746, 1036)

In [6]:
df.head()

,CONS_NO,FLAG,2014/01/01,2014/01/02,2014/01/03,2014/01/04,2014/01/05,2014/01/06,2014/01/07,2014/01/08,...,2016/10/22,2016/10/23,2016/10/24,2016/10/25,2016/10/26,2016/10/27,2016/10/28,2016/10/29,2016/10/30,2016/10/31
0,0387DD8A07E07FDA6271170F86AD9151,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,7.18,8.07,8.09,9.53,5.48,8.75,9.30,7.54,9.16,6.74
1,B32AC8CC6D5D805AC053557AB05F5343,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,12.81,15.12,17.26,14.91,19.59,20.79,17.95,19.26,14.46,11.72
2,EDFC78B07BA2908B3395C4EB2304665E,1,2.90,5.64,6.99,3.32,3.61,5.35,4.73,3.68,...,14.21,10.22,8.47,6.11,6.10,6.73,7.52,10.89,9.86,8.72
3,34C1954AA3703C4F8BD8EAEA7C4B7B83,1,0.11,0.11,0.25,0.27,0.21,0.20,0.14,0.34,...,1.85,3.36,1.45,2.12,1.55,1.18,1.13,1.34,1.26,1.40
4,768309B0EB11FD436CEE5ABFB84F4C0C,1,0.91,1.16,0.75,1.30,0.74,0.94,0.85,1.21,...,3.55,3.22,3.06,3.97,2.79,3.82,2.75,2.96,3.67,2.91


In [7]:
X = df.drop(["CONS_NO","FLAG"], axis=1).values
Y = df["FLAG"].values
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)



In [9]:
X_train.shape
df[df.isnull().sum(axis=1)>0]

,CONS_NO,FLAG,2014/01/01,2014/01/02,2014/01/03,2014/01/04,2014/01/05,2014/01/06,2014/01/07,2014/01/08,...,2016/10/22,2016/10/23,2016/10/24,2016/10/25,2016/10/26,2016/10/27,2016/10/28,2016/10/29,2016/10/30,2016/10/31


In [10]:
input_shape=(1034, 1)

In [11]:
# define CNN-LSTM model

#model1 CNN-LSTM Model

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(1034, 1)))
model.add(MaxPooling1D(pool_size=2))
# model.add(BatchNormalization())


model.add(LSTM(units=32, activation='relu'))
model.add(Flatten())

model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1032, 32)          128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 516, 32)          0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                        

In [12]:
# adam = Adam(learning_rate=0.001)
from keras import losses
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss=losses.mean_squared_error, optimizer="adam", metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=1, batch_size=128, callbacks=[chk], validation_data=(X_test,Y_test))

218/218 [==============================] - 60s 264ms/step - loss: nan - accuracy: 0.8462 - val_loss: nan - val_accuracy: 0.9175


In [13]:
# evaluate model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Accuracy: {accuracy}')

# make predictions
y_pred = model.predict(X_test)
y_pred


154/154 [==============================] - 7s 43ms/step - loss: nan - accuracy: 0.9175
Accuracy: 0.9175488352775574
154/154 [==============================] - 7s 46ms/step


array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)

In [14]:
# # define CNN-LSTM model
# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=2))

# model.add(Conv1D(filters=32, kernel_size=3, activation='relu')
# model.add(MaxPooling1D(pool_size=2))

# model.add(LSTM(units=32, activation='relu'))
# model.add(Dense(units=1, activation='sigmoid'))

# model.summary()

SyntaxError: invalid syntax (451446235.py, line 7)

In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(Y_test, y_pred))

In [ ]:
#model2 LSTM Model

model_LSTM = Sequential()
model_LSTM.add(LSTM(256, input_shape=(1034,1)))
model_LSTM.add(Dense(1, activation='sigmoid'))
model_LSTM.summary()

In [ ]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model_LSTM.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
history=model_LSTM.fit(X_train, Y_train, epochs=1, batch_size=256, callbacks=[chk], validation_data=(X_test,Y_test))

In [ ]:
# evaluate model
loss, accuracy = model_LSTM.evaluate(X_test, Y_test)
print(f'Accuracy: {accuracy}')

# make predictions
y_pred = model_LSTM.predict(X_test)
y_pred


In [ ]:
# evaluate the model
_, train_acc = model.evaluate(X_train, Y_train, verbose=0)
_, test_acc = model.evaluate(X_test, Y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
# plot accuracy during training
plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
# multilayer perceptron model for the two circles problem
from sklearn.datasets import make_circles
from keras.models import Sequential
from keras.layers import Dense

# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes=np.argmax(yhat_probs,axis=1)

#reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:]


# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(X_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, yhat_classes)
print('F1 score: %f' % f1)

